In [1]:
import pandas as pd
import plotly.express as px


data = pd.read_excel('../data/get_around_delay_analysis.xlsx')
for index, i in enumerate(data.columns):
    print(f"{i} --> {data.dtypes.iloc[index]}")
print()
print(data[data['delay_at_checkout_in_minutes'].notna()].head())

rental_id --> int64
car_id --> int64
checkin_type --> object
state --> object
delay_at_checkout_in_minutes --> float64
previous_ended_rental_id --> float64
time_delta_with_previous_rental_in_minutes --> float64

   rental_id  car_id checkin_type  state  delay_at_checkout_in_minutes  \
1     507750  269550       mobile  ended                         -81.0   
2     508131  359049      connect  ended                          70.0   
5     511626  398802       mobile  ended                        -203.0   
6     511639  370585      connect  ended                         -15.0   
7     512303  371242       mobile  ended                         -44.0   

   previous_ended_rental_id  time_delta_with_previous_rental_in_minutes  
1                       NaN                                         NaN  
2                       NaN                                         NaN  
5                       NaN                                         NaN  
6                  563782.0                    

In [4]:
print(data.isna().sum())
print(data["state"].value_counts())


rental_id                                         0
car_id                                            0
checkin_type                                      0
state                                             0
delay_at_checkout_in_minutes                   4964
previous_ended_rental_id                      19469
time_delta_with_previous_rental_in_minutes    19469
dtype: int64
state
ended       18045
canceled     3265
Name: count, dtype: int64


In [ ]:
print(f"Nombre de voiture : {data.value_counts(subset='car_id').sum()}")
# print(f"{data[data['']]}")
print("Distribution des ended et canceled :")
plot = px.bar(data.value_counts(subset='state').reset_index(),x='state',y='count')
plot.show()

Nombre de voiture : 21310
Distribution des ended et non ended :
